## Construct a Random Optimisation Problem

Creates a random optimisation problem with binary variables and equality constraints.

Usage : docplex.mp.model.Model m, soln = construct_optimization(9,4) numVariables, numConstraints
Example : m, soln = construct_optimization(9,4) <br>
Will return 
1. docplex.mp.model.Model m - a random optimisation problem with 9 binary variables and 4 equality constraints
2. Numpy single-dim array soln - the solution of m

#### TODO:
1. include integer variables
2. include continuous variables
3. include inequality constraints

In [10]:
import numpy as np
import docplex.mp
import random
from docplex.mp.model import Model
from docplex.mp.model_reader import ModelReader

In [11]:
def getRandomBinaryArray(k,n):
    print('k:',k)
    arr = np.zeros(n)
    arr[:k]  = 1
    np.random.shuffle(arr)
    return arr

def getLinearExpression(model, variableList, coeffList):
    from docplex.mp.linear import LinearExpr
    lnExpr = LinearExpr(model)
    if len(coeffList)>0:
        count=0
        for var in variableList:
            lnExpr.add(coeffList[count]*var)
            count=count+1
    else:
        for var in variableList:
            lnExpr.add(var)
        
    return (lnExpr)

def add_random_constraint(model, solution, vars):
    variables=[]
    coefficients=[]
    #solution_slice = solution[:random.randint(3,len(solution)-1)]
    count=0
    total_value=0
    for var in vars:
        #whether I am choosing this variable?
        #if (random.choice([True, False,True, False,True, False,False,True, False,True, False,False,True, False,False,True, False,True, False])):
        if (random.randint(0,100)>70):
            #This variable is chosen
            variables.append(vars[var])
            coeff=random.randint(-30,50) #Get a random coefficient
            print(var,vars[var], coeff, solution[count])
            coefficients.append(coeff)
            total_value = total_value + solution[count]*coeff
        count=count+1
    print(total_value)
    model.add_constraint(getLinearExpression(model,variables,coefficients) == total_value, 'randomCons')

def add_full_constraint(model, solution, vars):
    variables=[]
    count=0
    total_value=0
    for var in vars:
        variables.append(vars[var])
        total_value = total_value + solution[count]
        count=count+1
    print(total_value)
    model.add_constraint(getLinearExpression(model,variables,np.ones(len(variables))) == total_value,'allVarsCons')


def construct_optimization(numVariables, numConstraints):
    solution = getRandomBinaryArray(random.randint((int)(numVariables/2.2),(int)(numVariables/1.5)),numVariables)
    print(solution)
    #Intializing the variables
    vars = range(numVariables)
    dcpxMdl = Model('AMC CVRP Docplex Model')
    #Initializing our binary variable x_i,j
    x=dcpxMdl.binary_var_dict (vars,name='x')
    #Initializing the objectif function
    coeffs = [random.randint(-20,40) for i in vars]
    dcpxMdl.minimize(dcpxMdl.sum(coeffs[i]*x[i] for i in vars))
    ones = [i for i in range(len(solution)) if solution[i]>0]
    add_full_constraint(dcpxMdl, solution, x)
    for i in range(numConstraints-1):
        print('constraint: ',i)
        add_random_constraint(dcpxMdl, solution, x)
    #vars1 = range(random.randint(3,numVariables-1))
    #dcpxMdl.add_constraints(dcpxMdl.sum(x[j+i] for j in vars1  )==len(vars1)-i for i in range(numConstraints))
    
        
    return (dcpxMdl, solution)

def outputToFile(docplexModel):
    #Read Last File Number
    
    from docplex.mp.constants import int_probtype_to_string
    filename='..\\problemFiles\\999-RandomQUBO-'
    varText=''
    if docplexModel.number_of_binary_variables>0:
        varText='B%03d'%(m.number_of_binary_variables)+'-'
    if docplexModel.number_of_integer_variables>0:    
        varText=varText+'I%03d'%(m.number_of_integer_variables)+'-'
    if docplexModel.number_of_continuous_variables>0:    
        varText=varText+'C%03d'%(m.number_of_continuous_variables)+'-'
    cpx = m.get_cplex(do_raise=False)
    if cpx:
        cpx_probtype = cpx.get_problem_type()
        filename=filename+int_probtype_to_string(cpx_probtype)+'-'

    filename=filename+varText

    f = open("fileno.num", "r")
    try:
        number = int(f.readline())+1
    except:
        number = random.randint(1,999)
    f.close()
    
    #Create File Name
    filename = filename + '%03d'%number +'.lp'
    docplexModel.export_as_lp(path=filename)
    
    #Update Last Filenumber
    f = open("fileno.num", "w")
    f.write(str(number))
    f.close()
    print(number)

In [23]:
m, soln = construct_optimization(12,5)
m.print_information()

k: 8
[1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1.]
8.0
constraint:  0
5 x_5 35 1.0
6 x_6 23 1.0
8 x_8 -19 0.0
58.0
constraint:  1
2 x_2 1 0.0
4 x_4 -2 1.0
7 x_7 14 1.0
8 x_8 3 0.0
10 x_10 44 1.0
56.0
constraint:  2
3 x_3 39 0.0
7 x_7 42 1.0
9 x_9 5 1.0
10 x_10 -18 1.0
29.0
constraint:  3
2 x_2 -21 0.0
3 x_3 -30 0.0
6 x_6 43 1.0
8 x_8 5 0.0
11 x_11 -22 1.0
21.0
Model: AMC CVRP Docplex Model
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 5
   - linear=5
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [24]:
print(m.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: AMC CVRP Docplex Model

Minimize
 obj: 33 x_0 + 16 x_1 - 4 x_2 + 22 x_3 - 8 x_4 + 34 x_5 + 13 x_6 + 29 x_7
      + 14 x_9 - 20 x_10 + 37 x_11
Subject To
 allVarsCons: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 + x_10
              + x_11 = 8
 randomCons: 35 x_5 + 23 x_6 - 19 x_8 = 58
 randomCons#2: x_2 - 2 x_4 + 14 x_7 + 3 x_8 + 44 x_10 = 56
 randomCons#3: 39 x_3 + 42 x_7 + 5 x_9 - 18 x_10 = 29
 randomCons#4: - 21 x_2 - 30 x_3 + 43 x_6 + 5 x_8 - 22 x_11 = 21

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <= 1
 0 <= x_8 <= 1
 0 <= x_9 <= 1
 0 <= x_10 <= 1
 0 <= x_11 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9 x_10 x_11
End



In [25]:
outputToFile(m)

6
